In [ ]:
input_data = {
  "encrypted": True,
  "passwordProtected": True,
  "salt": "3kYbLHTv0PC6h4BiSy16Bg==",
  "kdfType": 0,
  "kdfIterations": 600000,
  "encKeyValidation_DO_NOT_EDIT": "2.mvGci287z4p1plmozNMaow==|lSItxCRWiRS8/g2eHTO8VSmKsc23B54UyMXQZkZJmh0WeXeZLKXnO2vv6eyRTb08|taiy8Sdc1Xi/5earAdf5PZeKq0JhhQumLYt7+xAWuGQ=",
  "data": "2.ptJf/HpC4d10umtorjeZ3w==|..."
}

In [6]:
import json
import base64
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding, hashes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from getpass import getpass

def decrypt_bitwarden_data(encrypted_data, master_password):

    # Extract the encryption parameters
    salt = base64.b64decode(encrypted_data['salt'])
    
    iterations = encrypted_data['kdfIterations']
    
    # Generate the master key using PBKDF2
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=iterations,
        backend=default_backend()
    )
    master_key = kdf.derive(master_password.encode())
    
    # Function to decrypt an encrypted string
    def decrypt_string(encrypted_str):
        if not encrypted_str.startswith('2.'):
            raise ValueError("Unsupported encryption version")
        
        encrypted_str = encrypted_str[2:]  # Remove version prefix
        parts = encrypted_str.split('|')
        if len(parts) != 2:
            raise ValueError("Invalid encrypted string format")
        
        iv = base64.b64decode(parts[0])
        ct = base64.b64decode(parts[1])
        
        cipher = Cipher(
            algorithms.AES(master_key),
            modes.CBC(iv),
            backend=default_backend()
        )
        
        decryptor = cipher.decryptor()
        padded_data = decryptor.update(ct) + decryptor.finalize()
        
        unpadder = padding.PKCS7(128).unpadder()
        return unpadder.update(padded_data) + unpadder.finalize()
    
    # Verify the master password using encKeyValidation
    try:
        validation_str = encrypted_data['encKeyValidation_DO_NOT_EDIT']
        decrypt_string(validation_str)
    except Exception:
        raise ValueError("Invalid master password")
    
    # Decrypt the main data
    decrypted_data = decrypt_string(encrypted_data['data'])
    return json.loads(decrypted_data)

def main():
    # Get the password from user
    password = getpass("Enter your master password: ")
    
    try:
        # Load the encrypted data
        encrypted_data = {
            "encrypted": True,
            "passwordProtected": True,
            "salt": input_data['salt'],
            "kdfType": input_data['kdfType'],
            "kdfIterations": input_data['kdfIterations'],
            "encKeyValidation_DO_NOT_EDIT": input_data['encKeyValidation_DO_NOT_EDIT'],
            "data": input_data['data']
        }
        
        # Decrypt the data
        decrypted_data = decrypt_bitwarden_data(encrypted_data, password)
        
        # Save to a file
        output_file = "decrypted_bitwarden_export.json"
        with open(output_file, 'w') as f:
            json.dump(decrypted_data, f, indent=2)
        
        print(f"Successfully decrypted! Saved to {output_file}")
        
    except ValueError as e:
        print(f"Error: {str(e)}")
    except Exception as e:
        print(f"Unexpected error: {str(e)}")

if __name__ == "__main__":
    main()

Unexpected error: name 'input_data' is not defined
